In [26]:
# update rmse 並記得重新啟動 Anaconda
# !pip install -U scikit-learn 

In [27]:
# Modules
import random
import pandas as pd
import numpy as np
from keras import models
from keras import layers
from keras import optimizers, metrics
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.layers.advanced_activations import LeakyReLU
import math
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
from scipy.stats import norm, skew #for some statistics
from sklearn.preprocessing import PowerTransformer

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import r2_score
from keras import backend as K

import warnings
warnings.filterwarnings('ignore')

In [28]:

def lr_rmse_ave(x,y):
    
    
    train_rmse=[]
    test_rmse=[]
    test_r2=[]
    
    for i in np.arange(10):
        x_train,x_test,y_train,y_test = train_test_split(x,y, random_state = i)
        lr = LinearRegression().fit(x_train,y_train)

        y_train_pred = lr.predict(x_train)
        y_test_pred = lr.predict(x_test)
        train_rmse.append(mean_squared_error(y_train, y_train_pred,squared=False))
        test_rmse.append(mean_squared_error(y_test, y_test_pred,squared=False))
        test_r2.append(r2_score(y_test,y_test_pred))
    
    train_rmse=np.array(train_rmse).mean()
    test_rmse=np.array(test_rmse).mean()
    test_r2=np.array(test_r2).mean()
    
#     print('train_rmse:', train_rmse)
#     print('test_rmse:', test_rmse)
#     print('test_r2:', test_r2)
    return train_rmse, test_rmse, test_r2

In [29]:
def skew_pro(data):
    columns=data.drop(['Predict'], axis=1).columns
    for col in columns: 
        if abs(data[col].skew()) >= 0.7: 
            pt = PowerTransformer() 
            d = pt.fit_transform(data[col].values.reshape(-1, 1)).flatten()
            data[col]=d
    X = data[columns]
    scaler = RobustScaler()
    data[columns] = scaler.fit_transform(X)
    
    return data

In [30]:
def r2_keras(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [31]:
def nn_ave(x,y):
    
    rmse=[]
    r2=[]

    for i in np.arange(10):
        X_train,X_val,Y_train,Y_val  = train_test_split(x, y, test_size=0.2, random_state=i)
        
        os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
        batch_size = 32
        epochs = 1000
        model = build_model()
        history = model.fit(
            X_train,
            Y_train,
            epochs = epochs,
            validation_data = (X_val, Y_val),
            batch_size = batch_size,
            shuffle = True,
            verbose=0
            )
        
        y_pred = model.predict(X_val)
        rmse.append(math.sqrt(mean_squared_error(Y_val , y_pred )))
        r2.append(r2_score(Y_val , y_pred ))
    
    test_rmse=np.array(rmse).mean()
    print('test_rmse_ave:',test_rmse)
    print(rmse)
    print('\n')
    test_r2=np.array(r2).mean()
    print('test_r2_ave:',test_r2)
    print(r2)

In [32]:
## 
n='Input_A2_017' # 改名稱
##
dire= ['./Data_Science_2020_imdb/BayesianRidge_Pre/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_1/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_2/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_3/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_4/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_5/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_6/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_7/']
print(n)
for d in dire:
    f_train_rmse=[]
    f_test_rmse=[]
    f_test_r2=[]
    print(d)
    data_d= d+n+'.csv'
    data= pd.read_csv(data_d,index_col=0).drop(['Number'], axis=1)
    y=data.Predict
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    f20=abs(numcorr['Predict']).sort_values(ascending=False).head(21).to_frame().index.to_numpy()[1:]
    fea_list=[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
    for i in fea_list:
        print('feature number:',i)
        x=skew_data[f20[:i]]
        tr_r,te_r, te_r2= lr_rmse_ave(x,y)
        print('train_rmse:',tr_r)
        print('test_rmse:',te_r)
        print('test_r2:',te_r2)
        f_train_rmse.append(tr_r)
        f_test_rmse.append(te_r)
        f_test_r2.append(te_r2)

    test_r=np.array(f_test_rmse)
    test_r2=np.array(f_test_r2)
    test_r_min= np.where(test_r == np.amin(test_r))
    test_r2_max= np.where(test_r2 == np.amax(test_r2))    
    print('Test_rmse_min = 1.fea_num:', fea_list[test_r_min[0][0]], ' 2.rmse:' ,np.amin(test_r))
    print('Test_r2_max = 1. fea_num:', fea_list[test_r2_max[0][0]],  ' 2.r2:', np.amax(test_r2))

Input_A2_017
./Data_Science_2020_imdb/BayesianRidge_Pre/
feature number: 2
train_rmse: 0.008826281643590467
test_rmse: 0.009732955071288676
test_r2: 0.5061291865763153
feature number: 3
train_rmse: 0.008446266606767224
test_rmse: 0.00940224743171986
test_r2: 0.5410683158193196
feature number: 4
train_rmse: 0.008348022503597662
test_rmse: 0.009201800887081354
test_r2: 0.5606734066564355
feature number: 5
train_rmse: 0.008300276103545664
test_rmse: 0.009091755270109353
test_r2: 0.5710755789644553
feature number: 6
train_rmse: 0.008253718457875855
test_rmse: 0.00912324424881122
test_r2: 0.5684217948165754
feature number: 7
train_rmse: 0.008242581202116187
test_rmse: 0.009169123800081096
test_r2: 0.5639367181492528
feature number: 8
train_rmse: 0.008199940685717985
test_rmse: 0.009123495837098535
test_r2: 0.5682560654195914
feature number: 9
train_rmse: 0.008193544370851255
test_rmse: 0.009162715556140063
test_r2: 0.5645350051272724
feature number: 10
train_rmse: 0.008186980377996287
test_

train_rmse: 0.008137133391634344
test_rmse: 0.00937398066505991
test_r2: 0.544250585720961
feature number: 15
train_rmse: 0.008100726944853087
test_rmse: 0.0093445187465306
test_r2: 0.5470513122015068
feature number: 16
train_rmse: 0.00809252816788663
test_rmse: 0.009393263979953773
test_r2: 0.5422513122320034
feature number: 17
train_rmse: 0.008064248281759579
test_rmse: 0.009389399300508586
test_r2: 0.5424351856893377
feature number: 18
train_rmse: 0.008028162061423314
test_rmse: 0.009405490846649631
test_r2: 0.5406978339289206
feature number: 19
train_rmse: 0.00797089614767559
test_rmse: 0.00934611676817286
test_r2: 0.5466623016239562
feature number: 20
train_rmse: 0.007928011313700314
test_rmse: 0.009322539239258843
test_r2: 0.5485767249104826
Test_rmse_min = 1.fea_num: 5  2.rmse: 0.009091755270109353
Test_r2_max = 1. fea_num: 5  2.r2: 0.5710755789644553
./Data_Science_2020_imdb/Data/BayesianRidge_Pre_4/
feature number: 2
train_rmse: 0.008826281643590467
test_rmse: 0.00973295507128

train_rmse: 0.008253718457875855
test_rmse: 0.00912324424881122
test_r2: 0.5684217948165754
feature number: 7
train_rmse: 0.008242581202116187
test_rmse: 0.009169123800081096
test_r2: 0.5639367181492528
feature number: 8
train_rmse: 0.008199940685717985
test_rmse: 0.009123495837098535
test_r2: 0.5682560654195914
feature number: 9
train_rmse: 0.008193544370851255
test_rmse: 0.009162715556140063
test_r2: 0.5645350051272724
feature number: 10
train_rmse: 0.008186980377996287
test_rmse: 0.009171395934359694
test_r2: 0.563735753398225
feature number: 11
train_rmse: 0.008169915964709906
test_rmse: 0.009225854018191069
test_r2: 0.5585099195742264
feature number: 12
train_rmse: 0.008152480604916202
test_rmse: 0.009285957472668699
test_r2: 0.552509023803839
feature number: 13
train_rmse: 0.008144792865049781
test_rmse: 0.009336577179336534
test_r2: 0.5477990760256952
feature number: 14
train_rmse: 0.008137133391634344
test_rmse: 0.00937398066505991
test_r2: 0.544250585720961
feature number: 15


# 可以修改第一層的神經元數量

In [33]:
#
def build_model():
    model = models.Sequential()
#
    model.add(layers.Dense(10, activation='sigmoid',input_shape=(x.shape[1],))) # 調這裡layers的數量
#
    model.add(layers.Dense(1, activation='linear'))
    model.compile(optimizer='adam', loss=metrics.mean_squared_error, metrics=[r2_keras])

    return model

In [34]:
fea_num=[5,6,7,8,9,10] # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/BayesianRidge_Pre/Input_A2_017.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=numcorr['Predict'].sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

feature number: 5
test_rmse_ave: 0.009355202646765225
[0.010163666598576955, 0.009654062886872773, 0.00928776277505415, 0.009401755169398927, 0.00896565827532682, 0.008275955425408635, 0.009616737779361308, 0.00992147173591184, 0.00850479734792609, 0.009760158473814751]


test_r2_ave: 0.5455245598596462
[0.5403627427482389, 0.5404622734103062, 0.5801513592332754, 0.5389580693873952, 0.5307704784408942, 0.6381627415821656, 0.5012402420634192, 0.47436978656759243, 0.6217369919580923, 0.48903091320508274]


feature number: 6
test_rmse_ave: 0.009266383333843706
[0.01029168566358376, 0.009794605479657546, 0.008731885869405213, 0.0101741503244218, 0.008071071531456633, 0.008532954185004497, 0.009529494417620752, 0.009444410503573662, 0.008870368356655587, 0.009223207007057614]


test_r2_ave: 0.5545948695922803
[0.5287108623622881, 0.5269851018572107, 0.6289037000493642, 0.46009313420602416, 0.619737624368194, 0.6153410638987575, 0.5102487231356263, 0.5237030146395024, 0.5885195155967654, 0.5

In [10]:
fea_num=[5,6,7,8,9,10]   # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_1/Input_A2_017.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=numcorr['Predict'].sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

feature number: 5
test_rmse_ave: 0.009276286027621214
[0.010155273740231958, 0.008936849783898778, 0.009127166585703292, 0.01049844059177746, 0.008164881767282561, 0.008447198579201816, 0.009160358778888048, 0.009566992646799539, 0.009344577751998453, 0.00936112005043025]


test_r2_ave: 0.5532900153984519
[0.5411215393059887, 0.6062053237266727, 0.5945451709120693, 0.42512669765109234, 0.6108466576298419, 0.6230338048985467, 0.5474559861205185, 0.5112587424013614, 0.5433480712412888, 0.52995816009714]


feature number: 6
test_rmse_ave: 0.009103601100457305
[0.009674511548808475, 0.009635883846183933, 0.009578965314183013, 0.009357537409874855, 0.007843039228889365, 0.008283943825652637, 0.008822428243042964, 0.009804939051249259, 0.008601242622902603, 0.009433519913785954]


test_r2_ave: 0.5703455787015901
[0.583540763529043, 0.5421913049462264, 0.5534113057959318, 0.5432845597481548, 0.6409212217375944, 0.637463874628748, 0.5802292952724961, 0.4866448556845321, 0.6131092654361693, 0.5

In [11]:
fea_num=[5,6,7,8,9,10]  # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_2/Input_A2_017.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=numcorr['Predict'].sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

feature number: 5
test_rmse_ave: 0.00936380670426882
[0.009700924655410495, 0.009159713410041553, 0.009372358860554705, 0.00979126247429086, 0.008015717828829231, 0.008596493508741889, 0.009985461764464581, 0.009861396739628731, 0.009845605330591475, 0.00930913247013466]


test_r2_ave: 0.5445751464229435
[0.5812636463662264, 0.5863198372211614, 0.5724682809046637, 0.49996553897741824, 0.62493563323366, 0.6095911287451719, 0.4622602225887291, 0.4807159483289245, 0.49306674985171683, 0.5351644780117621]


feature number: 6
test_rmse_ave: 0.00911715440086123
[0.00998133494375482, 0.008901772989663569, 0.008987108675101703, 0.010019132481151468, 0.007963102000487422, 0.008166485782739109, 0.009804162469248782, 0.009537976437818549, 0.008389895140144226, 0.009420573088502658]


test_r2_ave: 0.5678511548469851
[0.5567061943905575, 0.6092905153338879, 0.6068932422289528, 0.47642031372915306, 0.6298433792159587, 0.6476717886385746, 0.48160971249973605, 0.5142189020304779, 0.6318888280508421, 0

In [12]:
fea_num=[5,6,7,8,9,10] # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_3/Input_A2_017.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=numcorr['Predict'].sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

feature number: 5
test_rmse_ave: 0.009197740031280075
[0.009771889044071582, 0.009039896617158833, 0.00941501534181357, 0.009899123066729519, 0.008122601424930743, 0.008293580797031611, 0.008792083744513807, 0.010835631298294868, 0.008923779637739574, 0.008883799340516654]


test_r2_ave: 0.5597505442751444
[0.5751149424801816, 0.5970716253399888, 0.5685677687446018, 0.4888880947731039, 0.6148665408811254, 0.636619884729205, 0.5831119089121435, 0.37304472665236843, 0.5835492897273773, 0.5766706605113476]


feature number: 6
test_rmse_ave: 0.009348718791152887
[0.01018124263846584, 0.009714144456805124, 0.009393754628585379, 0.00949269246846375, 0.008312604357776443, 0.009496328064839247, 0.00842280585290296, 0.009611348157606468, 0.009184878079481788, 0.009677389206601874]


test_r2_ave: 0.5474822170540852
[0.5387716657765964, 0.5347246555154581, 0.570514064106449, 0.529996196053633, 0.5966378107581083, 0.5235817192516978, 0.6173960604753621, 0.5067163281410734, 0.5588231412789175, 0.49

In [13]:
fea_num=[5,6,7,8,9,10]  # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_4/Input_A2_017.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=numcorr['Predict'].sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

feature number: 5
test_rmse_ave: 0.009112432936598264
[0.009639934366147924, 0.009384675929249683, 0.00896133373354866, 0.009902952314144934, 0.008240129428027898, 0.007913331203983413, 0.008549131477416608, 0.009563209796532553, 0.008858797831667297, 0.010110833285263687]


test_r2_ave: 0.5681442557975487
[0.586512335558216, 0.5657503422988818, 0.6091448616087702, 0.48849259459138505, 0.6036407199993976, 0.669177008486588, 0.6058333745180808, 0.5116451688457891, 0.5895922872963274, 0.45165386477205094]


feature number: 6
test_rmse_ave: 0.009042018881378011
[0.009384741174395023, 0.009014651724158189, 0.008934069992630625, 0.009558501801486608, 0.00847512173990049, 0.008216477291440746, 0.009451182758215472, 0.009295976118558449, 0.008824628218535528, 0.009264837994458978]


test_r2_ave: 0.5755618205596638
[0.6081146730277158, 0.5993189255228247, 0.6115194999174254, 0.5234568812484257, 0.5807115927198339, 0.6433449998629546, 0.5182650235903139, 0.538556938226999, 0.5927521804995899, 0

In [14]:
fea_num=[5,6,7,8,9,10]  # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_5/Input_A2_017.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=numcorr['Predict'].sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

feature number: 5
test_rmse_ave: 0.009574359163285999
[0.010466924938467017, 0.00904296015569979, 0.009507153148157345, 0.009841773764721066, 0.008831651081854696, 0.00908190495731198, 0.009639388760401667, 0.011303100859486385, 0.008606254894343603, 0.00942247907241643]


test_r2_ave: 0.5226251475765209
[0.5125246910140369, 0.5967984815600356, 0.5600822324291876, 0.4947930630039804, 0.5446925360233352, 0.5642565137272328, 0.49888794695891914, 0.3177817587279994, 0.6126582221420988, 0.5237760301783825]


feature number: 6
test_rmse_ave: 0.009660487560143753
[0.010469207758622723, 0.008997412310675051, 0.009175433506445724, 0.010356771917965676, 0.008345746020834143, 0.009350084976432381, 0.009924775836305272, 0.011199692145174878, 0.008665990044441567, 0.010119761084540116]


test_r2_ave: 0.5132433570804567
[0.5123120326030328, 0.6008499667765802, 0.5902455228699899, 0.44053699426622506, 0.5934150560282437, 0.5381423789995146, 0.46877651100619866, 0.330207482231494, 0.6072625608445017,

In [15]:
fea_num=[5,6,7,8,9,10] # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_6/Input_A2_017.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=numcorr['Predict'].sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

feature number: 5
test_rmse_ave: 0.009356053841588736
[0.00981127321451539, 0.01022186278138385, 0.009706156688087069, 0.009414629812216913, 0.00803081752183135, 0.008381654889011088, 0.009546310458258173, 0.009378313037717674, 0.009512847907622498, 0.009556672105243365]


test_r2_ave: 0.5463784344998535
[0.571683166514152, 0.4848176002560298, 0.5414727835726303, 0.5376945150310493, 0.62352123938415, 0.6288610383397029, 0.5085187375816669, 0.5303464910065747, 0.5267539051319767, 0.5101148681806023]


feature number: 6
test_rmse_ave: 0.009297274480294156
[0.009468312737435583, 0.009680333121722543, 0.009779226455670262, 0.009180389076798162, 0.008095558295466005, 0.009642041561139309, 0.009470351546935188, 0.009484518401325652, 0.008377037974957797, 0.009794975631491062]


test_r2_ave: 0.5514646998630811
[0.6011040819130973, 0.5379579207295613, 0.5345430394506547, 0.5604131166890429, 0.617426773843933, 0.5088490406267882, 0.516308942399522, 0.5196490116851622, 0.633016193674729, 0.48537

In [16]:
fea_num=[5,6,7,8,9,10] # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_7/Input_A2_017.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=numcorr['Predict'].sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

feature number: 5
test_rmse_ave: 0.009066331192156201
[0.009742474924977597, 0.008916577112327143, 0.01050262917900709, 0.00930625800618842, 0.007836801595809193, 0.00811470868895789, 0.0084980736406327, 0.009609885899899069, 0.008940017608442848, 0.009195885265320053]


test_r2_ave: 0.5736517410414146
[0.5776689645818376, 0.6079898933598147, 0.46313315185328985, 0.5482764552057965, 0.6414921511781008, 0.6521252832614797, 0.6105274649758775, 0.5068664117651136, 0.5820323390270696, 0.5464052952057663]


feature number: 6
test_rmse_ave: 0.009652080199929974
[0.01096472994458586, 0.010273936325936752, 0.009949464371348847, 0.009516126878011604, 0.008812017951942063, 0.0095572505590557, 0.009777953306773694, 0.009891770000931941, 0.008676671473448689, 0.009100881187264574]


test_r2_ave: 0.5178554828289557
[0.4650535827762623, 0.47955521159529146, 0.5181965279916585, 0.5276727547317998, 0.546714621021766, 0.517449308586946, 0.4843776011447266, 0.4775122156010364, 0.6062938123542494, 0.5557